In [1]:
print("test")

test


In [2]:
import mido

In [3]:
midi_file = mido.MidiFile('./one_note_quantize.mid')

In [4]:
midi_file

MidiFile(type=1, ticks_per_beat=690, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='one_note_quantize', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('set_tempo', tempo=384615, time=0),
    MetaMessage('marker', text='nombreux anglele', time=1610),
    MetaMessage('marker', text='', time=493580),
    MetaMessage('time_signature', numerator=3, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=459770),
    MetaMessage('key_signature', key='Dm', time=0),
    MetaMessage('set_tempo', tempo=333334, time=0),
    MetaMessage('set_tempo', tempo=333334, time=16560),
    MetaMessage('set_tempo', tempo=333334, time=11385),
    MetaMessage('set_tempo', tempo=335300, time=122),
    MetaMessage('set_tempo', tempo=337176, time=123),
    MetaMessage('set_tempo', tempo=339074, time=122),
    MetaMessage('set_tempo', tempo=340993, time=123),
    MetaMessage('set_tempo',

In [5]:
for msg in midi_file:
    if msg.__class__ == mido.Message:
        print(msg)

note_on channel=0 note=62 velocity=102 time=0
note_on channel=0 note=62 velocity=0 time=0.16666699999999998
note_on channel=0 note=65 velocity=102 time=0
note_on channel=0 note=65 velocity=0 time=0.16666699999999998
note_on channel=0 note=74 velocity=102 time=0
note_on channel=0 note=74 velocity=0 time=0.6666679999999999
note_on channel=0 note=62 velocity=102 time=0
note_on channel=0 note=62 velocity=0 time=0.16666699999999998
note_on channel=0 note=65 velocity=102 time=0
note_on channel=0 note=65 velocity=0 time=0.16666699999999998
note_on channel=0 note=74 velocity=102 time=0
note_on channel=0 note=74 velocity=0 time=0.6666679999999999
note_on channel=0 note=76 velocity=102 time=0
note_on channel=0 note=76 velocity=0 time=0.5000009999999999
note_on channel=0 note=77 velocity=102 time=0
note_on channel=0 note=77 velocity=0 time=0.16666699999999998
note_on channel=0 note=76 velocity=102 time=0
note_on channel=0 note=76 velocity=0 time=0.16666699999999998
note_on channel=0 note=77 veloc

In [6]:
import umidiparser as midi

In [7]:
mf = midi.MidiFile("./one_note_quantize.mid", buffer_size=0)

In [8]:
for event in mf:
    print(event)

track_name delta[miditicks]=0 data=b'one_n...' delta[usec]=0 name=one_note_quantize status=3
time_signature delta[miditicks]=0 data=b'\x04\x02\x18\x08' delta[usec]=0 clocks_per_click=24 denominator=4 notated_32nd_notes_per_beat=8 numerator=4 status=88
set_tempo delta[miditicks]=0 data=b'\x05\xdeg' delta[usec]=0 status=81 tempo=384615
track_name delta[miditicks]=0 data=b'Instr...' delta[usec]=0 name=Instrument Track 01 status=3
marker delta[miditicks]=1610 data=b'nombr...' delta[usec]=897435 status=6 text=nombreux anglele
marker delta[miditicks]=493580 data=b'' delta[usec]=275127930 status=6 text=
time_signature delta[miditicks]=459770 data=b'\x03\x02\x18\x08' delta[usec]=256281795 clocks_per_click=24 denominator=4 notated_32nd_notes_per_beat=8 numerator=3 status=88
key_signature delta[miditicks]=0 data=b'\xff\x01' delta[usec]=0 key=Dm status=89
set_tempo delta[miditicks]=0 data=b'\x05\x16\x16' delta[usec]=0 status=81 tempo=333334
set_tempo delta[miditicks]=16560 data=b'\x05\x16\x16' de

In [9]:
from typing import List, Dict



def event_to_dict(me: midi.MidiEvent) -> Dict[str, int]:
    if (me.status == midi.NOTE_OFF) or (me.status == midi.NOTE_ON) and (me.velocity == 0):
        status = 0
    elif me.status == midi.NOTE_ON:
        status = 1
    else:
        status = None
    return {
        "status": status,
        "note": me.note,
        "delta": me.delta_us,
    }

def midi_to_list(mf: midi.MidiFile) -> List[Dict[str,float]]:
    midi_list: Dict[str, int] = [
        event_to_dict(ev) for ev in mf if (ev.status == midi.NOTE_OFF) or (ev.status == midi.NOTE_ON)
    ]
    return midi_list

In [10]:
list_dic = midi_to_list(mf)

In [11]:
def delta_to_time(midi_list: List[Dict[str,float]]) -> List[Dict[str,float]]:
    prev_delta = 0
    for e in midi_list:
        time = prev_delta + e["delta"]
        prev_delta = time
        e["time"] = time
    return midi_list

In [12]:
import pandas as pd
import numpy as np

def note_to_rad(n: int):
    return (np.pi/6)*(5-n)%(2*np.pi)


def list_to_point(l: List[Dict[str,float]]):
    point_list = []
    for i_e in range(len(l)):
        event = l[i_e]
        if i_e == 0:
            point_list.append({
                "status": event["status"],
                "note": event["note"],
                "x": 0,
                "y": 0,
                "time": 0,
            })
        else :
            if event["status"] == 1:
                prev_point = point_list[-1]
                prev_rad = note_to_rad(prev_point["note"])
                prev_x = prev_point["x"]
                prev_y = prev_point["y"]
                point_list.append({
                    "status": event["status"],
                    "note": event["note"],
                    "x": np.cos(prev_rad)*event["delta"] + prev_x,
                    "y": np.sin(prev_rad)*event["delta"] + prev_y,
                    "time": event["time"],
                })
            else:
                for inv in range(-1, -len(point_list)-1, -1):
                    if point_list[inv]["note"] == event["note"]:
                        start_event = point_list[inv]
                        prev_x = start_event["x"]
                        prev_y = start_event["y"]
                        rad = note_to_rad(event["note"])
                        delta = event["time"] - start_event["time"]
                        point_list.append({
                            "status": event["status"],
                            "note": event["note"],
                            "x": np.cos(rad)*delta + prev_x,
                            "y": np.sin(rad)*delta + prev_y,
                            "time": event["time"],
                        })
                        break
    return point_list

        

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def point_to_segment(l):
    l_segments = []
    for i in range(len(l)):
        point = l[i]
        if point["status"] == 1:
            for k in range(i+1,len(l)):
                end_point=l[k]
                if (point["note"] == end_point["note"]) and end_point["status"] == 0:
                    l_segments.append({
                        "note": point["note"],
                        "x1": point["x"],
                        "y1": point["y"],
                        "x2": end_point["x"],
                        "y2": end_point["y"],
                        "lenght": end_point["time"] - point["time"],
                    })
                    break
    return l_segments

In [ ]:
l_segment = point_to_segment(list_to_point(delta_to_time(list_dic)))

In [ ]:
df = pd.DataFrame(l_segment)

In [ ]:
df

,note,x1,y1,x2,y2,color,note_name
0,60,0,0,1,1,"rgb(234, 71, 71)",C
1,61,1,1,2,0,"rgb(234, 153, 71)",C#
2,62,2,0,3,1,"rgb(234, 234, 71)",D
3,72,3,1,4,0,"rgb(237, 94, 94)",C
4,73,4,0,5,1,"rgb(237, 165, 94)",C#
5,74,5,1,6,0,"rgb(237, 237, 94)",D
6,84,6,0,7,1,"rgb(239, 117, 117)",C
7,85,7,1,8,0,"rgb(239, 178, 117)",C#
8,86,8,0,9,1,"rgb(239, 239, 117)",D
9,60,9,1,10,0,"rgb(234, 71, 71)",C


In [ ]:
import plotly.graph_objects as go
import plotly.colors


# Normalize note values
min_note, max_note = df['note'].min(), df['note'].max()

# Use Plotly's color scale
colorscale = 'Viridis'
colors = plotly.colors.sample_colorscale(colorscale, (df['note'] - min_note) / (max_note - min_note))

# Initialize figure
fig = go.Figure()

# Plot each segment with color based on note
for i, row in df.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['x1'], row['x2']],
        y=[row['y1'], row['y2']],
        mode='lines',
        line=dict(color=colors[i], width=3),
        name=f'Note {row["note"]}',
        showlegend=False
    ))

# Add a color bar
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(
        size=10,
        color=list(df['note']),
        colorscale=colorscale,
        cmin=min_note, cmax=max_note,
        colorbar=dict(title="Note")
    ),
    hoverinfo='none'
))

# Layout adjustments
fig.update_layout(
    title="Line Segments Colored by Integer Note",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_dark",
    width=1200,
    height=800
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import colorsys

# Music notation for the 12 pitch classes
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

# Define color map function
def midi_note_to_color(note):
    """Map a MIDI note to a color using HSL with octave-based brightness."""
    semitone = note % 12
    octave = note // 12 - 1
    hue = semitone * (360 / 12) / 360  # Normalized hue
    base_lightness = 0.4
    lightness_step = 0.05
    lightness = min(base_lightness + octave * lightness_step, 0.9)
    saturation = 0.8
    r, g, b = colorsys.hls_to_rgb(hue, lightness, saturation)
    return f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})', note_names[semitone]

# Assign colors and note names
df[['color', 'note_name']] = df['note'].apply(lambda n: pd.Series(midi_note_to_color(n)))

# Initialize Plotly figure
fig = go.Figure()

# Track which notes have been added to the legend
legend_added = set()

# Plot each segment as a single line segment with hover info
for _, row in df.iterrows():
    note_label = f'{row["note_name"]}{row["note"] // 12 - 1}'
    fig.add_trace(go.Scatter(
        x=[row['x1'], row['x2'], None],  # Add None to prevent connecting lines between segments
        y=[row['y1'], row['y2'], None],
        mode='lines',
        line=dict(color=row['color'], width=4),
        name=note_label if note_label not in legend_added else None,  # Only show legend once per note
        hoverinfo='text',
        text=f'Note: {note_label} ({row["note"]})',
        legendgroup=note_label,  # Group all segments of the same note
        showlegend=note_label not in legend_added,  # Show legend only for the first occurrence
    ))
    legend_added.add(note_label)  # Mark this note as added to the legend

# Layout adjustments
fig.update_layout(
    title="Line Segments Colored by MIDI Note (Unique Legend Entries)",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_dark",
    width=1200,
    height=1200,
    showlegend=True,
    legend_traceorder="grouped",
    legend_itemclick=False,
    legend_itemdoubleclick=False,
)

fig.update_traces(
    selector=dict(type='scatter'),
    visible=True
)

fig.show()
